In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch
import os
import pandas as pd

c:\Users\cristian.sanchezp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\cristian.sanchezp\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cristian.sanchezp\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
df = pd.read_csv('../data/raw/idcloud-content.csv')

In [5]:
passage_embeddings = list(model.encode(df['content'].to_list(), show_progress_bar=True))


Batches: 100%|██████████| 22/22 [00:24<00:00,  1.13s/it]


In [12]:
# Define a function to find relevant news articles based on a given query
def find_relevant_info(query):
    # Encode the query using the sentence transformer model
    query_embedding = model.encode(query)
    # Print the shape of the query embedding
    query_embedding.shape

    # Calculate the cosine similarity between the query embedding and the passage embeddings
    similarities = util.cos_sim(query_embedding, passage_embeddings)

    # Find the indices of the top 3 most similar passages
    top_indicies = torch.topk(similarities.flatten(), 3).indices

    # Get the top 3 relevant passages by slicing the summaries at 200 characters and adding an ellipsis
    top_relevant_passages = [df.iloc[x.item()]['title'][:200] + "..." for x in top_indicies]

    # Return the top 3 relevant passages
    return top_relevant_passages

In [13]:
find_relevant_info("Journeys")

['Success node :: ForgeRock Identity Cloud Docs...',
 'Authentication nodes and journeys :: ForgeRock Identity Cloud Docs...',
 'Self-service :: ForgeRock Identity Cloud Docs...']